In [1]:
# Importa a biblioteca pandas 
import pandas as pd

# Importa datetime e timedelta para verificar se há gaps de tempo nos datasets preparados
from datetime import datetime, timedelta

# Importa a biblioteca os
import os

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, OrdinalEncoder

#Para as figuras
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

#Importa a biblioteca Numpy
import numpy as np

# Importa as bibliotecas para os cálculos do desvio padrao
from scipy.stats import norm
import statistics

# Get the Data & Explore the Data
### Após primeira execução e montagem dos datasets, ir para item "2. Explore the Data"

# 1 Get the Data
## 1.1 HTTP - Prepare the Data
### df_http
* df_http recebe 3 novas colunas, **'down'**, **'up'** e **'vis'** indicando a ocorrência **(1)** o não **(0)** de **download**, **upload** e **visit**.

### df_h_1hora 
* Cria-se o dataset **df_1hora**, que representa a consolidação de downlods, uploads e visits de cada usuário no intervalo de 1 hora 
* df_1hora recebe 3 novas colunas, **'hour'**, indicando a hora cheia (0-23), **'dow'** indicando o dia da semana


#### Não mais usado daqui pra baixo
* df_3horas
* coluna **user_n**, indicando um número único atribuído ao usuário **(ordinal_encoder)**.
* df_1hora_3meses - Gerado on the fly na execução dos algoritmos, quando do carregamento dos datasets.
Criam-se 4 novos datasets:
A partir de df_1hora:
* df_1hora_3meses: 3 meses de dados contínuos a partir de df_1hora
* df_1hora_1mes: 1 mes de dados contínuos a partir de df_1hora

In [2]:
%%time
# Carregamento do dataset original, só na primeira execução!
#Carrega o dataset fazendo atribuição de tipos para diminuir consumo de memória
df_http = pd.read_csv("D:/OneDrive/ML/Datasets/Cert/data/r6.2/http.csv", usecols=['date', 'user', 'pc', 'url', 'activity'], infer_datetime_format=True,
                     parse_dates=['date'], dtype={"user":"category", "pc":"category","url":"category","activity":"category"})

Wall time: 7min 6s


* df_http recebe 3 novas colunas, **'down'**, **'up'** e **'vis'** indicando a ocorrência **(1)** o não **(0)** de **download**, **upload** e **visit**.

In [7]:
# Só na primeira execução!
# Adiciona coluna de valores 0 ou 1 conforme coluna 'activity', para
# depois consolidar as somas das atividades nessas colunas
df_http['down'] = np.where((df_http['activity']== 'WWW Download'),1,0)
df_http['up'] = np.where((df_http['activity']== 'WWW Upload'),1,0)
df_http['vis'] = np.where((df_http['activity']== 'WWW Visit'),1,0)

# Salva o Dataset em arquivo.
# Nas próximas rodadas, basta carregar o arquivo "df_http_file.pkl". 
df_http.to_pickle("df_http_file.pkl")


### NAO UTILILZADO!!. Apenas para consulta se precisar
# quando há mais condições, utiliza-se o np.select()
#df_http['act'] = np.select([(df_http['activity']== 'WWW Download'),
#                            (df_http['activity']== 'WWW Vist'),
#                           (df_http['activity']== 'WWW Upload')],[-1,0,1])

* Criam-se novo datasets, **df_1hora**, que representa a consolidação de downlods, uploads e visits de cada usuário no intervalo de **1 hora**

In [8]:
# Rodar somente na primeira execução
#Agrupamento para consolidar dados a cada 1 hora, criando baseline temporal por usuário
df_h_1hora = df_http.groupby(['user',pd.Grouper(key='date',freq='1H')])['down','up','vis'].sum().reset_index().sort_values(['user','date'], ascending=[True, True])


### NAO UTILIZADO
#Agrupamento para consolidar dados a cada 1 hora, criando baseline temporal por usuário
#df_3horas = df_http.groupby(['user',pd.Grouper(key='date',freq='3H')])['down','up','vis'].sum().reset_index().sort_values(['user','date'], ascending=[True, True])

C:\Users\albmo\AppData\Local\Temp\ipykernel_8808\1813446097.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_h_1hora = df_http.groupby(['user',pd.Grouper(key='date',freq='1H')])['down','up','vis'].sum().reset_index().sort_values(['user','date'], ascending=[True, True])


* **df_1hora** recebe 2 novas colunas, **'hour'**, indicando a hora cheia (0-23), **'dow'** indicando o dia da semana 

In [9]:
# Rodar somente na primeira execução
#Criando a coluna para a hora dos eventos
df_h_1hora['hour'] = df_h_1hora['date'].dt.hour

#Criando a coluna dow para dia da semana
df_h_1hora['dow'] = df_h_1hora['date'].dt.dayofweek

## Limpeza das linhas com valores de zero para down up vis
df_h_1hora.drop(df_h_1hora[(df_h_1hora['down'] == 0) & (df_h_1hora['up'] == 0) & (df_h_1hora['vis'] == 0)].index, axis=0, inplace=True)

### Grava em arquivo a versão sem zeros
df_h_1hora.to_pickle("df_h_1hora_file.pkl")
### Carrega do arquivo
df_h_1hora = pd.read_pickle("df_h_1hora_file.pkl")

In [ ]:
## NAO MAIS UTILIZADO - Adição da coluna user_n

#Converte usuários em categorias numéricas em df_1hora
#ordinal_encoder = OrdinalEncoder()
#user_encoded = ordinal_encoder.fit_transform(df_1hora[['user']])
#len(np.unique(user_encoded))
#ordinal_encoder.categories_
#user_enc_df = pd.DataFrame(user_encoded)
#user_enc_df.columns = ['user_n']
#user_enc_df.shape, df_1hora.shape
#df_1hora = pd.concat([df_1hora.reset_index(drop=True), 
#                             user_enc_df.reset_index(drop=True)], axis=1)
#ordinal_encoder.transform([['YJP2791']])

#Converte usuários em categorias numéricas em df_3horas
#ordinal_encoder = OrdinalEncoder()
#user_encoded = ordinal_encoder.fit_transform(df_3horas[['user']])
#user_enc_df = pd.DataFrame(user_encoded)
#user_enc_df.columns = ['user_n']
#df_3horas = pd.concat([df_3horas.reset_index(drop=True), 
#                             user_enc_df.reset_index(drop=True)], axis=1)

In [ ]:
## NAO MAIS UTILIZADO

## A partir de df_1hora_3meses e df_1hora_1mes
#Estabelecendo dataset com 3 meses de dados para referência
#mask = (df_1hora['date'] >= '2010-01-02') & (df_1hora['date'] <= '2010-04-02')
#df_1hora_3meses = df_1hora.loc[mask]

#Estabelecendo dataset com 1 mes de dados para referência
#mask = (df_1hora['date'] >= '2010-01-02') & (df_1hora['date'] <= '2010-02-02')
#df_1hora_1mes = df_1hora.loc[mask]

#######  Fatia o dataset principal em 8 meses a partir do treinamento
#Estabelecendo dataset com 8 meses de dados a partir do treinamento
#mask = (df_1hora['date'] >= '2010-04-03') & (df_1hora['date'] <= '2010-12-03')
#df_1hora_8meses = df_1hora.loc[mask]


## A partir de df_3horas_3meses e df_3horas_1mes
#Estabelecendo dataset com 3 meses de dados para referência
#mask = (df_3horas['date'] >= '2010-01-02') & (df_3horas['date'] <= '2010-04-02')
#df_3horas_3meses = df_3horas.loc[mask]

#Estabelecendo dataset com 1 mes de dados para referência
#mask = (df_3horas['date'] >= '2010-01-02') & (df_3horas['date'] <= '2010-02-02')
#df_3horas_1mes = df_3horas.loc[mask]

# Salva os Datasets em arquivos.
# Nas próximas rodadas, basta carregar os arquivos "df_1h_3mfile.pkl" e "df_1h_1mfile.pkl", "df_3h_3mfile.pkl", "df_3h_1mfile.pkl".

#df_1hora_3meses.to_pickle("df_1h_3mfile.pkl")
#df_1hora_1mes.to_pickle("df_1h_1mfile.pkl")
#df_1hora_8meses.to_pickle("df_1h_8mfile.pkl")
#df_3horas_3meses.to_pickle("df_3h_3mfile.pkl")
#df_3horas_1mes.to_pickle("df_3h_1mfile.pkl")

* Verifica se há gaps de horários para cada usuário na base df_1hora. Vai auxiliar caso seja aplicado a LSTM, onde se espera eventos temporais sem gaps.

In [ ]:
## Não usado
# Teste para verificar se há gaps
# Demorado para todo o dataset
#for user_loop in df_h_1hora['user']:
#    deltas = df_h_1hora[df_h_1hora['user'] == user_loop]['date'].diff()[1:]
#    gaps = deltas[deltas > timedelta(hours=1)]
#    if len(gaps) > 0:
#        print("Usuario ", user_loop, "position", deltas[deltas > timedelta(hours=1)].index) 

#from datetime import datetime, timedelta
#deltas = df_user['date'].diff()[1:]
#gaps = deltas[deltas > timedelta(hours=1)]

# Se houver gaps...
#r = pd.date_range(start=df_1hora_1mes.date.min(), end=df_1hora_1mes.date.max())
#r = pd.date_range(start=df_1hora_1mes['date'].dt.date.min(), end=df_1hora_1mes['date'].dt.date.max())
#df_1hora_1mes.set_index('date').reindex(r).fillna(0.0).rename_axis('dt').reset_index()

## 1.2 USB - Prepare the Data

In [17]:
%%time
# Carregamento do dataset original, só na primeira execução!
#Carrega o dataset fazendo atribuição de tipos para diminuir consumo de memória
df_usb = pd.read_csv("D:/OneDrive/ML/Datasets/Cert/data/r6.2/file.csv", 
                      usecols=['date','user','pc','activity','to_removable_media','from_removable_media'], 
                      infer_datetime_format=True,parse_dates=['date'],
                      dtype={"user":"category", "pc":"category","activity":"category",
                             "to_removable_media":"int32","from_removable_media":"int32"})

df_usb.rename(columns={'to_removable_media':'trm',
                       'from_removable_media':'frm'},
              inplace = True)

# Só na primeira execução!
# Adiciona coluna de valores 0 ou 1 conforme coluna 'activity'
df_usb['open'] = np.where((df_usb['activity']== 'File Open'),1,0)
df_usb['write'] = np.where((df_usb['activity']== 'File Write'),1,0)
df_usb['copy'] = np.where((df_usb['activity']== 'File Copy'),1,0)
df_usb['delete'] = np.where((df_usb['activity']== 'File Delete'),1,0)

#Retira as colunas 'activity' e 'pc'
df_usb.drop(['activity','pc'], axis='columns', inplace=True)

## Salva em arquivo para uso posterior
df_usb.to_pickle("df_usb_file.pkl")

## Carrega arquivo 
df_usb = pd.read_pickle("df_usb_file.pkl")

# Rodar somente na primeira execução
#Agrupamento para consolidar dados a cada 1 hora, criando baseline temporal por usuário
#df_usb_1hora = df_usb.groupby(['user',pd.Grouper(key='date',freq='1H')])[\
#               'open','write','copy','delete','to_removable_media',\
#               'from_removable_media'].sum().reset_index().sort_values(['user','date'], ascending=[True, True])

# Rodar somente na primeira execução
#Criando a coluna para a hora dos eventos
#df_usb_1hora['hour'] = df_usb_1hora['date'].dt.hour

#Criando a coluna dow para dia da semana
#df_usb_1hora['dow'] = df_usb_1hora['date'].dt.dayofweek

## A partir de df_1hora_3meses e df_1hora_1mes
#Estabelecendo dataset com 3 meses de dados para referência
#mask = (df_usb_1hora['date'] >= '2010-01-02') & (df_usb_1hora['date'] <= '2010-04-02')
#df_usb_1hora_3meses = df_usb_1hora.loc[mask]

#######  Fatia o dataset principal em 8 meses a partir do treinamento
#Estabelecendo dataset com 8 meses de dados a partir do treinamento
#mask = (df_usb_1hora['date'] >= '2010-04-03') & (df_usb_1hora['date'] <= '2010-12-03')
#df_usb_1hora_8meses = df_usb_1hora.loc[mask]

# Salva os Datasets em arquivos.
# Nas próximas rodadas, basta carregar o arquivo "df_usb_1h_file.pkl" 
#df_usb_1hora.to_pickle("df_usb_1h_file.pkl")
#df_usb_1hora_3meses.to_pickle("df_usb_1h_3mfile.pkl")
#df_usb_1hora_8meses.to_pickle("df_usb_1h_8mfile.pkl")

Wall time: 10.4 s


In [18]:
df_usb

,date,user,trm,frm,open,write,copy,delete
0,2010-01-02 07:19:41,SDH2394,0,1,1,0,0,0
1,2010-01-02 07:21:30,SDH2394,1,0,0,1,0,0
2,2010-01-02 07:22:11,SDH2394,0,1,0,0,1,0
3,2010-01-02 07:24:06,SDH2394,1,0,0,1,0,0
4,2010-01-02 07:24:45,SDH2394,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
2014878,2011-05-31 23:36:00,ABD3426,0,1,0,0,0,1
2014879,2011-05-31 23:47:51,QAH0048,0,1,1,0,0,0
2014880,2011-05-31 23:48:23,QAH0048,0,1,1,0,0,0
2014881,2011-05-31 23:49:13,QAH0048,0,1,1,0,0,0


## 1.3 Device - Prepare the Data

In [11]:
%%time
# Carregamento do dataset original, só na primeira execução!
#Carrega o dataset fazendo atribuição de tipos para diminuir consumo de memória
df_device = pd.read_csv("D:/OneDrive/ML/Datasets/Cert/data/r6.2/device.csv", 
           usecols=['date','user','pc','activity'], 
           infer_datetime_format=True,parse_dates=['date'],
           dtype={"user":"category", "pc":"category","activity":"category"})

# Só na primeira execução!
# Adiciona coluna de valores 0 ou 1 conforme coluna 'activity'
df_device['conn'] = np.where((df_device['activity']== 'Connect'),1,0)
df_device['disc'] = np.where((df_device['activity']== 'Disconnect'),1,0)

#Retira as colunas 'activity' e 'pc'
df_device.drop(['activity','pc'], axis='columns', inplace=True)

## Salva em arquivo para uso posterior
df_device.to_pickle("df_device_file.pkl")

## Carrega arquivo 
df_device = pd.read_pickle("df_device_file.pkl")

Wall time: 5.31 s


In [12]:
df_device

,date,user,conn,disc
0,2010-01-02 07:17:18,SDH2394,1,0
1,2010-01-02 07:22:42,JKS2444,1,0
2,2010-01-02 07:31:42,CBA1023,1,0
3,2010-01-02 07:33:28,GNT0221,1,0
4,2010-01-02 07:33:55,JKS2444,0,1
...,...,...,...,...
1551823,2011-05-31 23:02:16,WSA1765,0,1
1551824,2011-05-31 23:31:40,ABD3426,1,0
1551825,2011-05-31 23:36:13,ABD3426,0,1
1551826,2011-05-31 23:46:23,QAH0048,1,0


## 1.4 Logon - Prepare the Data

In [9]:
%%time
# Carregamento do dataset original, só na primeira execução!
#Carrega o dataset fazendo atribuição de tipos para diminuir consumo de memória
df_logon = pd.read_csv("D:/OneDrive/ML/Datasets/Cert/data/r6.2/logon.csv", 
           usecols=['date','user','pc','activity'], 
           infer_datetime_format=True,parse_dates=['date'],
           dtype={"user":"category", "pc":"category","activity":"category"})

# Só na primeira execução!
# Adiciona coluna de valores 0 ou 1 conforme coluna 'activity'
df_logon['logon'] = np.where((df_logon['activity']== 'Logon'),1,0)
df_logon['logoff'] = np.where((df_logon['activity']== 'Logoff'),1,0)

#Retira as colunas 'activity' e 'pc'
df_logon.drop(['activity','pc'], axis='columns', inplace=True)

## Salva em arquivo para uso posterior
df_logon.to_pickle("df_logon_file.pkl")

## Carrega arquivo 
df_logon = pd.read_pickle("df_logon_file.pkl")

Wall time: 11.2 s


In [10]:
df_logon

,date,user,logon,logoff
0,2010-01-02 02:19:18,DNS1758,1,0
1,2010-01-02 02:31:12,DNS1758,0,1
2,2010-01-02 02:34:02,DNS1758,1,0
3,2010-01-02 02:53:30,DNS1758,0,1
4,2010-01-02 04:07:31,DNS1758,1,0
...,...,...,...,...
3530280,2011-06-01 05:39:10,QAH1315,0,1
3530281,2011-06-01 06:07:37,HMY0235,0,1
3530282,2011-06-01 06:13:11,CRM0139,0,1
3530283,2011-06-01 06:49:27,ABM3641,0,1


## 1.5 Concatenação dos datasets http e usb

In [7]:
%%time
df_http = pd.read_pickle("df_http_file.pkl")
df_usb = pd.read_pickle("df_usb_file.pkl")

df_http.drop(['pc','url','activity'], axis='columns', inplace=True)

df_http_usb = pd.concat([df_http, df_usb],ignore_index=True)
df_http_usb.fillna(0, inplace=True)
df_http_usb.sort_values(['user','date'], ascending=[True, True], inplace=True)
df_http_usb["down"]   = df_http_usb['down'].astype('int32')
df_http_usb["up"]     = df_http_usb['up'].astype('int32')
df_http_usb["vis"]    = df_http_usb['vis'].astype('int32')
df_http_usb["trm"]    = df_http_usb['trm'].astype('int32')
df_http_usb["frm"]    = df_http_usb['frm'].astype('int32')
df_http_usb["open"]   = df_http_usb['open'].astype('int32')
df_http_usb["write"]  = df_http_usb['write'].astype('int32')
df_http_usb["copy"]   = df_http_usb['copy'].astype('int32')
df_http_usb["delete"] = df_http_usb['delete'].astype('int32')

### Grava em arquivo
df_http_usb.to_pickle("df_http_usb_file.pkl")

### Carrega do arquivo
df_http_usb = pd.read_pickle("df_http_usb_file.pkl")

Wall time: 9min 55s


In [4]:
%%time

# Gera dataset dh_hu_1hora que é a consolidação das atividades a cada hora.

# Rodar somente na primeira execução
#Agrupamento para consolidar dados a cada 1 hora, criando baseline temporal por usuário
df_hu_1hora = df_http_usb.groupby(['user',pd.Grouper(key='date',freq='1H')])['down','up','vis','trm',
              'frm','open','write','copy','delete'].sum().reset_index().sort_values(['user','date'], ascending=[True, True])

#Criando a coluna para a hora dos eventos
df_hu_1hora['hour'] = df_hu_1hora['date'].dt.hour

#Criando a coluna dow para dia da semana
df_hu_1hora['dow'] = df_hu_1hora['date'].dt.dayofweek

### Grava em arquivo
df_hu_1hora.to_pickle("df_hu_1hora_file.pkl")

### Carrega do arquivo
df_hu_1hora = pd.read_pickle("df_hu_1hora_file.pkl")

<timed exec>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Wall time: 2min 48s


In [6]:
df_http_usb = pd.read_pickle("df_http_usb_file.pkl")

In [7]:
%%time

# Gera dataset dfz_hu_1hora que é a consolidação das atividades a cada hora
#incluindo zeros nos horarios sem uso para cada 'user'. Equivalente a padding.

# Rodar somente na primeira execução
#Agrupamento para consolidar dados a cada 1 hora, criando baseline temporal por usuário

dfz_hu_1hora = df_http_usb.set_index('date').groupby('user').resample('H')[
    'down','up','vis','trm','frm','open','write','copy','delete'
 ].sum().reset_index().sort_values(['user','date'], ascending=[True, True])

#Criando a coluna para a hora dos eventos
dfz_hu_1hora['hour'] = dfz_hu_1hora['date'].dt.hour

#Criando a coluna dow para dia da semana
dfz_hu_1hora['dow'] = dfz_hu_1hora['date'].dt.dayofweek

### Grava em arquivo
dfz_hu_1hora.to_pickle("dfz_hu_1hora_file.pkl")

### Carrega do arquivo
dfz_hu_1hora = pd.read_pickle("dfz_hu_1hora_file.pkl")

Wall time: 1min 55s


In [8]:
df_hu_1hora

,user,date,down,up,vis,trm,frm,open,write,copy,delete,hour,dow
0,AAB0162,2010-01-04 07:00:00,0,0,3,0,0,0,0,0,0,7,0
1,AAB0162,2010-01-04 08:00:00,0,0,25,0,0,0,0,0,0,8,0
2,AAB0162,2010-01-04 09:00:00,0,0,4,0,0,0,0,0,0,9,0
3,AAB0162,2010-01-04 10:00:00,0,0,11,0,0,0,0,0,0,10,0
4,AAB0162,2010-01-04 11:00:00,0,0,18,0,0,0,0,0,0,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12660336,ZZO2997,2011-05-31 10:00:00,0,0,15,0,0,0,0,0,0,10,1
12660337,ZZO2997,2011-05-31 11:00:00,0,0,22,0,0,0,0,0,0,11,1
12660338,ZZO2997,2011-05-31 12:00:00,0,0,13,0,0,0,0,0,0,12,1
12660339,ZZO2997,2011-05-31 13:00:00,0,0,25,0,0,0,0,0,0,13,1


In [8]:
dfz_hu_1hora

,user,date,down,up,vis,trm,frm,open,write,copy,delete,hour,dow
0,AAB0162,2010-01-04 07:00:00,0,0,3,0,0,0,0,0,0,7,0
1,AAB0162,2010-01-04 08:00:00,0,0,25,0,0,0,0,0,0,8,0
2,AAB0162,2010-01-04 09:00:00,0,0,4,0,0,0,0,0,0,9,0
3,AAB0162,2010-01-04 10:00:00,0,0,11,0,0,0,0,0,0,10,0
4,AAB0162,2010-01-04 11:00:00,0,0,18,0,0,0,0,0,0,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47073377,ZZO2997,2011-05-31 10:00:00,0,0,15,0,0,0,0,0,0,10,1
47073378,ZZO2997,2011-05-31 11:00:00,0,0,22,0,0,0,0,0,0,11,1
47073379,ZZO2997,2011-05-31 12:00:00,0,0,13,0,0,0,0,0,0,12,1
47073380,ZZO2997,2011-05-31 13:00:00,0,0,25,0,0,0,0,0,0,13,1


## 1.6 Concatenação dos datasets http, device e usb

In [6]:
%%time
df_http   = pd.read_pickle("df_http_file.pkl")
df_device = pd.read_pickle("df_device_file.pkl")
df_usb    = pd.read_pickle("df_usb_file.pkl")

df_http.drop(['pc','url','activity'], axis='columns', inplace=True)

df_hud = pd.concat([df_http, df_device],ignore_index=True)
df_hud = pd.concat([df_hud, df_usb],ignore_index=True)
df_hud.fillna(0, inplace=True)
df_hud.sort_values(['user','date'], ascending=[True, True], inplace=True)
df_hud["down"]   = df_hud['down'].astype('int32')
df_hud["up"]     = df_hud['up'].astype('int32')
df_hud["vis"]    = df_hud['vis'].astype('int32')
df_hud["conn"]   = df_hud['conn'].astype('int32')
df_hud["disc"]   = df_hud['disc'].astype('int32')
df_hud["trm"]    = df_hud['trm'].astype('int32')
df_hud["frm"]    = df_hud['frm'].astype('int32')
df_hud["open"]   = df_hud['open'].astype('int32')
df_hud["write"]  = df_hud['write'].astype('int32')
df_hud["copy"]   = df_hud['copy'].astype('int32')
df_hud["delete"] = df_hud['delete'].astype('int32')

### Grava em arquivo
df_hud.to_pickle("df_hud_file.pkl")

### Carrega do arquivo
df_hud = pd.read_pickle("df_hud_file.pkl")

Wall time: 10min 22s


In [13]:
df_hud

,date,user,down,up,vis,conn,disc,trm,frm,open,write,copy,delete
40975,2010-01-04 07:46:30,AAB0162,0,0,1,0,0,0,0,0,0,0,0
40998,2010-01-04 07:46:36,AAB0162,0,0,1,0,0,0,0,0,0,0,0
41091,2010-01-04 07:46:51,AAB0162,0,0,1,0,0,0,0,0,0,0,0
48031,2010-01-04 08:03:10,AAB0162,0,0,1,0,0,0,0,0,0,0,0
48310,2010-01-04 08:03:46,AAB0162,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116929890,2011-05-31 14:36:40,ZZO2997,0,0,1,0,0,0,0,0,0,0,0
116929963,2011-05-31 14:36:48,ZZO2997,0,0,1,0,0,0,0,0,0,0,0
116930624,2011-05-31 14:38:03,ZZO2997,0,0,1,0,0,0,0,0,0,0,0
116932092,2011-05-31 14:40:39,ZZO2997,0,0,1,0,0,0,0,0,0,0,0


In [3]:
%%time

# Gera dataset df_hud_1hora que é a consolidação das atividades a cada hora.

# Rodar somente na primeira execução
#Agrupamento para consolidar dados a cada 1 hora, criando baseline temporal por usuário
df_hud_1hora = df_hud.groupby(
    ['user',pd.Grouper(key='date',freq='1H')]
)['down','up','vis','conn','disc','trm','frm','open','write','copy','delete'
 ].sum().reset_index().sort_values(['user','date'], ascending=[True, True])

#Criando a coluna para a hora dos eventos
df_hud_1hora['hour'] = df_hud_1hora['date'].dt.hour

#Criando a coluna dow para dia da semana
df_hud_1hora['dow'] = df_hud_1hora['date'].dt.dayofweek

### Grava em arquivo
df_hud_1hora.to_pickle("df_hud_1hora_file.pkl")

### Carrega do arquivo
df_hud_1hora = pd.read_pickle("df_hud_1hora_file.pkl")

<timed exec>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Wall time: 3min 1s


In [2]:
df_hud = pd.read_pickle("df_hud_file.pkl")

In [4]:
%%time

# Gera dataset dfz_hud_1hora que é a consolidação das atividades a cada hora
#incluindo zeros nos horarios sem uso para cada 'user'. Equivalente a padding.

# Rodar somente na primeira execução
#Agrupamento para consolidar dados a cada 1 hora, criando baseline temporal por usuário

dfz_hud_1hora = df_hud.set_index('date').groupby('user').resample('H')[
    'down','up','vis','conn','disc','trm','frm','open','write','copy','delete'
 ].sum().reset_index().sort_values(['user','date'], ascending=[True, True])

#Criando a coluna para a hora dos eventos
dfz_hud_1hora['hour'] = dfz_hud_1hora['date'].dt.hour

#Criando a coluna dow para dia da semana
dfz_hud_1hora['dow'] = dfz_hud_1hora['date'].dt.dayofweek

### Grava em arquivo
dfz_hud_1hora.to_pickle("dfz_hud_1hora_file.pkl")

### Carrega do arquivo
dfz_hud_1hora = pd.read_pickle("dfz_hud_1hora_file.pkl")

Wall time: 2min 44s


In [4]:
df_hud_1hora

,user,date,down,up,vis,conn,disc,trm,frm,open,write,copy,delete,hour,dow
0,AAB0162,2010-01-04 07:00:00,0,0,3,0,0,0,0,0,0,0,0,7,0
1,AAB0162,2010-01-04 08:00:00,0,0,25,0,0,0,0,0,0,0,0,8,0
2,AAB0162,2010-01-04 09:00:00,0,0,4,0,0,0,0,0,0,0,0,9,0
3,AAB0162,2010-01-04 10:00:00,0,0,11,0,0,0,0,0,0,0,0,10,0
4,AAB0162,2010-01-04 11:00:00,0,0,18,0,0,0,0,0,0,0,0,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12688954,ZZO2997,2011-05-31 10:00:00,0,0,15,0,0,0,0,0,0,0,0,10,1
12688955,ZZO2997,2011-05-31 11:00:00,0,0,22,0,0,0,0,0,0,0,0,11,1
12688956,ZZO2997,2011-05-31 12:00:00,0,0,13,0,0,0,0,0,0,0,0,12,1
12688957,ZZO2997,2011-05-31 13:00:00,0,0,25,0,0,0,0,0,0,0,0,13,1


In [5]:
df_hud_1hora.corr()

,down,up,vis,conn,disc,trm,frm,open,write,copy,delete,hour,dow
down,1.000000,0.589900,-0.045963,0.045101,0.044833,0.110392,0.026899,0.055714,0.186045,0.017466,0.017590,-0.001103,-0.001722
up,0.589900,1.000000,-0.033471,0.034855,0.034960,0.069536,0.027358,0.073959,0.113652,0.014625,0.014427,0.000433,-0.001391
vis,-0.045963,-0.033471,1.000000,0.012589,0.012675,-0.004119,-0.000882,0.027920,-0.004311,-0.002092,-0.000062,-0.013859,0.004553
conn,0.045101,0.034855,0.012589,1.000000,0.843063,0.435137,0.466066,0.288876,0.335714,0.446884,0.407789,-0.009070,0.015466
disc,0.044833,0.034960,0.012675,0.843063,1.000000,0.434175,0.464924,0.288050,0.334453,0.446156,0.406604,0.010029,0.015515
trm,0.110392,0.069536,-0.004119,0.435137,0.434175,1.000000,0.707332,0.432976,0.808593,0.823436,0.617122,-0.000285,0.007905
frm,0.026899,0.027358,-0.000882,0.466066,0.464924,0.707332,1.000000,0.553330,0.510480,0.870962,0.872064,-0.000346,0.009355
open,0.055714,0.073959,0.027920,0.288876,0.288050,0.432976,0.553330,1.000000,0.323433,0.458851,0.424751,-0.003716,0.008489
write,0.186045,0.113652,-0.004311,0.335714,0.334453,0.808593,0.510480,0.323433,1.000000,0.474005,0.440219,-0.000925,0.005406
copy,0.017466,0.014625,-0.002092,0.446884,0.446156,0.823436,0.870962,0.458851,0.474005,1.000000,0.668369,-0.000047,0.008961


In [5]:
dfz_hud_1hora

,user,date,down,up,vis,conn,disc,trm,frm,open,write,copy,delete,hour,dow
0,AAB0162,2010-01-04 07:00:00,0,0,3,0,0,0,0,0,0,0,0,7,0
1,AAB0162,2010-01-04 08:00:00,0,0,25,0,0,0,0,0,0,0,0,8,0
2,AAB0162,2010-01-04 09:00:00,0,0,4,0,0,0,0,0,0,0,0,9,0
3,AAB0162,2010-01-04 10:00:00,0,0,11,0,0,0,0,0,0,0,0,10,0
4,AAB0162,2010-01-04 11:00:00,0,0,18,0,0,0,0,0,0,0,0,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47073428,ZZO2997,2011-05-31 10:00:00,0,0,15,0,0,0,0,0,0,0,0,10,1
47073429,ZZO2997,2011-05-31 11:00:00,0,0,22,0,0,0,0,0,0,0,0,11,1
47073430,ZZO2997,2011-05-31 12:00:00,0,0,13,0,0,0,0,0,0,0,0,12,1
47073431,ZZO2997,2011-05-31 13:00:00,0,0,25,0,0,0,0,0,0,0,0,13,1


## 1.7 Concatenação dos datasets logon, http, device e usb

In [2]:
%%time
df_logon  = df_logon = pd.read_pickle("df_logon_file.pkl")
df_http   = pd.read_pickle("df_http_file.pkl")
df_device = pd.read_pickle("df_device_file.pkl")
df_usb    = pd.read_pickle("df_usb_file.pkl")

df_http.drop(['pc','url','activity'], axis='columns', inplace=True)

df_lhud = pd.concat([df_logon, df_http],ignore_index=True)
df_lhud = pd.concat([df_lhud, df_device],ignore_index=True)
df_lhud = pd.concat([df_lhud, df_usb],ignore_index=True)
df_lhud.fillna(0, inplace=True)
df_lhud.sort_values(['user','date'], ascending=[True, True], inplace=True)
df_lhud["logon"]  = df_lhud['logon'].astype('int32')
df_lhud["logoff"] = df_lhud['logoff'].astype('int32')
df_lhud["down"]   = df_lhud['down'].astype('int32')
df_lhud["up"]     = df_lhud['up'].astype('int32')
df_lhud["vis"]    = df_lhud['vis'].astype('int32')
df_lhud["conn"]   = df_lhud['conn'].astype('int32')
df_lhud["disc"]   = df_lhud['disc'].astype('int32')
df_lhud["trm"]    = df_lhud['trm'].astype('int32')
df_lhud["frm"]    = df_lhud['frm'].astype('int32')
df_lhud["open"]   = df_lhud['open'].astype('int32')
df_lhud["write"]  = df_lhud['write'].astype('int32')
df_lhud["copy"]   = df_lhud['copy'].astype('int32')
df_lhud["delete"] = df_lhud['delete'].astype('int32')

### Grava em arquivo
df_lhud.to_pickle("df_lhud_file.pkl")

### Carrega do arquivo
df_lhud = pd.read_pickle("df_lhud_file.pkl")

Wall time: 11min 2s


In [3]:
%%time

# Gera dataset df_lhud_1hora que é a consolidação das atividades a cada hora.

# Rodar somente na primeira execução
#Agrupamento para consolidar dados a cada 1 hora, criando baseline temporal por usuário
df_lhud_1hora = df_lhud.groupby(
    ['user',pd.Grouper(key='date',freq='1H')]
)['logon','logoff','down','up','vis','conn','disc','trm','frm','open','write','copy','delete'
 ].sum().reset_index().sort_values(['user','date'], ascending=[True, True])

#Criando a coluna para a hora dos eventos
df_lhud_1hora['hour'] = df_lhud_1hora['date'].dt.hour

#Criando a coluna dow para dia da semana
df_lhud_1hora['dow'] = df_lhud_1hora['date'].dt.dayofweek

### Grava em arquivo
df_lhud_1hora.to_pickle("df_lhud_1hora_file.pkl")

### Carrega do arquivo
df_lhud_1hora = pd.read_pickle("df_lhud_1hora_file.pkl")

<timed exec>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Wall time: 3min 36s
